In [ ]:
import random
import pyttsx3
import time
import threading
import tkinter as tk
from tkinter import messagebox

def speak(text):
    engine = pyttsx3.init()
    engine.say(text)
    engine.runAndWait()

def generate_question(difficulty):
    formulas = [
        ("What is the area of a circle with radius 7? (Use π=3.14)", 3.14 * 7 * 7),
        ("What is the Pythagorean theorem formula?", "a^2 + b^2 = c^2"),
        ("What is the perimeter of a square with side length 5?", 4 * 5),
        ("What is the quadratic formula?", "x = (-b ± sqrt(b^2 - 4ac)) / 2a"),
        ("What is the volume of a cube with side length 3?", 3**3)
    ]
    
    if random.random() < 0.3:  # 30% chance to ask a formula-based question
        return random.choice(formulas)
    
    if difficulty == "Easy":
        num1, num2 = random.randint(1, 10), random.randint(1, 10)
    elif difficulty == "Medium":
        num1, num2 = random.randint(10, 50), random.randint(10, 50)
    else:
        num1, num2 = random.randint(50, 100), random.randint(50, 100)
    
    operation = random.choice(['+', '-', '*'])
    if operation == '+':
        answer = num1 + num2
    elif operation == '-':
        answer = num1 - num2
    else:
        answer = num1 * num2
    
    return f"{num1} {operation} {num2}", answer

def chatbot_response(user_input):
    responses = {
        "hi": "Hello! Ready to test your math skills?",
        "how are you": "I'm just a bot, but I'm here to challenge you with math!",
        "bye": "Goodbye! Keep practicing your math skills!",
        "help": "Solve math problems! You have limited time to answer."
    }
    response = responses.get(user_input.lower(), "I don't understand that. Try 'hi' or 'help'.")
    speak(response)
    return response

def get_answer_with_timeout(question, timeout=10):
    answer = [None]
    def user_input():
        answer[0] = input(f"What is {question}? ")
    
    thread = threading.Thread(target=user_input)
    thread.start()
    thread.join(timeout)
    if thread.is_alive():
        print("⏳ Time's up! No answer recorded.")
        speak("Time's up! No answer recorded.")
        return None
    return answer[0]

def math_quiz():
    score, lives = 0, 3
    streak, multiplier = 0, 1
    rounds = 5
    difficulty = input("🎮 Choose difficulty (Easy/Medium/Hard): ").capitalize()
    speak(f"You chose {difficulty} mode. Let's begin!")
    print(f"🎯 You chose {difficulty} mode. Let's begin!")
    
    for _ in range(rounds):
        question, correct_answer = generate_question(difficulty)
        speak(f"You have 10 seconds. What is {question}?")
        print(f"⏳ You have 10 seconds to answer: {question}")
        
        user_answer = get_answer_with_timeout(question)
        if user_answer is None:
            lives -= 1
            print(f"❌ You lost a life! ❤️ Lives remaining: {lives}")
            speak(f"You lost a life! Lives remaining: {lives}")
            if lives == 0:
                break
            continue
        
        if user_answer.lower() in ["hi", "how are you", "bye", "help"]:
            print(chatbot_response(user_answer))
            continue
        
        if isinstance(correct_answer, str):  # Formula-based question
            if user_answer.strip().lower() == correct_answer.lower():
                print("✅ Correct! You remembered the formula!")
                speak("Correct! You remembered the formula!")
                score += 2  # Extra points for formula questions
            else:
                print(f"❌ Wrong! The correct answer is: {correct_answer}")
                speak(f"Wrong! The correct answer is: {correct_answer}")
                lives -= 1
        elif user_answer.isdigit() and int(user_answer) == correct_answer:
            streak += 1
            if streak >= 3:
                multiplier = 2
            score += 1 * multiplier
            print(f"✅ Correct! 🔥 Streak: {streak}, 🎯 Score: {score}")
            speak(f"Correct! Streak: {streak}, Score: {score}")
        else:
            streak, multiplier = 0, 1
            print(f"❌ Wrong! The correct answer was {correct_answer}.")
            speak(f"Wrong! The correct answer was {correct_answer}.")
            lives -= 1
            if lives == 0:
                break
    
    final_message = f"🎉 Game over! Your final score is {score}."
    print(final_message)
    speak(final_message)
    if score >= rounds:
        print("🏆 Congratulations! You are a math master!")
        speak("Congratulations! You are a math master!")
    elif score > 0:
        print("👍 Good effort! Keep practicing!")
        speak("Good effort! Keep practicing!")
    else:
        print("💡 Don't give up! Try again!")
        speak("Don't give up! Try again!")

if __name__ == "__main__":
    math_quiz()
